In [1]:
## Importing all dependencies 
import pandas as pd
import datetime
import warnings

 
warnings.filterwarnings("ignore")


#### Raw Material Data Preprocessing: Independent Step

This stage encompasses preprocessing tasks and additional calculations, including the conversion of costs into current exchange rate."

In [2]:
###### RM data Processing #######

##### Reading Default/ Template data #######

################ RM Budgeted Cost Input Files ##################################

#Read BUdget Data
df_nfc = pd.read_excel(r"PROTEIN_COGS_FORECAST-Q4+2024_BUDGET-TP3_V16.01.xlsx", sheet_name='NFC_BOM',usecols="A:I,AW:BH", skiprows = 3)
df_kfc = pd.read_excel(r"PROTEIN_COGS_FORECAST-Q4+2024_BUDGET-TP3_V16.01.xlsx", sheet_name='KFC_BOM',usecols="A:I,AW:BH", skiprows = 3)
df_gfc = pd.read_excel(r"PROTEIN_COGS_FORECAST-Q4+2024_BUDGET-TP3_V16.01.xlsx", sheet_name='GFC_BOM',usecols="A:I,AW:BH", skiprows = 3)

################## Item Cost Data ###########################################

# Read Item Cost data
##### Logic: Only Dec-23 Item Cost/ Consumption Costs will be used 
df_gfc_i = pd.read_excel(r'ITEM COST GFC DEC-23.xls')
df_kfc_i = pd.read_excel(r'ITEM COST KFC DEC-23.xls')
df_nfc_i = pd.read_excel(r'ITEM COST NFC DEC-23.xls')

########################### Forecsted Cost Data ###############################

# Read Forecasted Cost Data
# df_frct =pd.read_excel(r"Protein Commodities Impact_110324 (1) (1).xlsx",sheet_name='Dashboard',usecols="A:AA",skiprows=1)
### Updating with RM 2023 price
# df_frct =pd.read_excel(r"24_04_RM Prices_2023 v2.xlsx",sheet_name='Dashboard',usecols="A:AA",skiprows=1)
df_frct =pd.read_excel(r"23_04_2024 RM Prices for 2023 Backtest v4.xlsx",sheet_name='Dashboard',usecols="A:AA",skiprows=1)
df_frct

,Status,ID Code,Code,Description,2024 Volume,Entity,Subcategory,Category,Responsible,Management,...,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00
0,NaN,1001101127GFC,1001101127,CHICKEN TRIMMING,NaN,GFC,NaN,NaN,NaN,NaN,...,1.937972,1.877684,1.812675,1.802160,1.721858,1.867997,1.810403,2.058041,2.113722,2.145325
1,NaN,7012100126GFC,7012100126,WHITE MILKY BIG BAGS,NaN,GFC,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,NaN,2012106251GFC,2012106251,LABEL,NaN,GFC,NaN,NaN,NaN,NaN,...,0.009536,0.009536,0.009533,0.009533,0.009533,0.009531,0.009531,0.009531,0.009531,0.009531
3,NaN,2012114588GFC,2012114588,ROYAL CO-PACKING CARTONS,NaN,GFC,NaN,NaN,NaN,NaN,...,0.881423,0.881423,0.881423,0.881423,0.881423,0.881423,0.881423,0.839405,0.833862,0.697000
4,NaN,2012104429GFC,2012104429,TAPE,NaN,GFC,NaN,NaN,NaN,NaN,...,4.878639,4.878704,4.878492,4.878476,4.878476,4.878437,4.878435,4.878435,4.879073,4.879505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,NaN,2011114801NFCC,2011114801,CHICKEN BURGER BAG N3,NaN,NFCC,NaN,NaN,NaN,NaN,...,0.047658,0.047658,0.049157,0.049157,0.046049,0.045475,0.045475,0.047485,0.048416,0.048944
1482,NaN,2011114914NFCC,2011114914,FALAFEL AMER. ROLL N3,NaN,NFCC,NaN,NaN,NaN,NaN,...,0.125824,0.125824,0.124177,0.124177,0.124177,0.124177,0.124177,0.128557,0.129006,0.129341
1483,NaN,2011995254NFCC,2011995254,ROYAL BREADED REGULAR CHICKEN FILLET ROLL,NaN,NFCC,NaN,NaN,NaN,NaN,...,0.117333,0.117333,0.117333,0.117333,0.117333,0.117333,0.117333,0.117333,0.117333,0.117333
1484,NaN,4001370230NFCC,4001370230,Chicken Nuggets 400G (18*1*400Gm),NaN,NFCC,NaN,NaN,NaN,NaN,...,18.378569,16.806015,14.143222,15.959817,15.479329,15.612539,15.613236,15.742542,16.090526,16.199821


In [3]:
####### One-time data (Entity Master Data) ##### 
# Define the data for the entity mapping
data = {
    'Entity': ['GFI', 'NFCM', 'GFC', 'KFC', 'NFCC'],
    'Factory code': ['GFC', 'NFC', 'GFC', 'KFC', 'NFC']
}

# Create a pandas DataFrame
df_entity_mapping = pd.DataFrame(data)
# df_entity_mapping


In [4]:
'''' List out functions '''
##### Exchange Rates As inputs #####
def get_conversion_rates():
    """
    Get conversion rates from user input with default values.

    Returns:
    - dict: A dictionary containing conversion rates for each factory.
    """
    use_default = input("Do you want to use default values? The following Conversion rates will be applied 'KFC': 0.306001, 'NFC': 3.75, 'GFC': 3.6725 (Y/N): ").upper()

    if use_default == 'Y':
        # conversion_rates = {'KFC': 0.306001, 'NFC': 3.75, 'GFC': 3.6725}
        conversion_rates = {'KFC': 3.26796317659, 'NFC': 0.26666666666, 'GFC': 0.2722940776}
    else:
        conversion_rates = {}
        conversion_rates['GFC'] = float(input("Enter conversion rate for GFC (1 AED= ? USD): "))
        conversion_rates['KFC'] = float(input("Enter conversion rate for KFC (1 KWD = ? USD): "))
        conversion_rates['NFC'] = float(input("Enter conversion rate for NFC (1 SAR = ? USD): "))
    
    return conversion_rates

# Function to convert date string to datetime
def convert_to_datetime(date_string):
    date_format = date_format_dict.get(date_string)
    if date_format:
        return datetime.datetime.strptime(date_string, date_format)
    else:
        return None

In [5]:
#### RM Budgeted Cost Processing ####

try:

    #Add FactoryCode column
    df_nfc.insert(df_nfc.columns.get_loc('Required Qty'), 'FactCode', 'NFC')
    df_kfc.insert(df_kfc.columns.get_loc('Required Qty'), 'FactCode', 'KFC')
    df_gfc.insert(df_gfc.columns.get_loc('Required Qty'), 'FactCode', 'GFC')

    ### Take exchange rates from the input data

    conversion_rates = get_conversion_rates()
    print('Currently used Exchange rates are :', conversion_rates)

    df_nfc.insert(df_nfc.columns.get_loc('FactCode'), 'Con_rate', conversion_rates['NFC'])
    df_kfc.insert(df_kfc.columns.get_loc('FactCode'), 'Con_rate', conversion_rates['KFC'])
    df_gfc.insert(df_gfc.columns.get_loc('FactCode'), 'Con_rate', conversion_rates['GFC'])

    print ('Exchnage rates added')

    ## Unpivot tables 
    # NFC unpivot
    up_df_nfc = pd.melt(df_nfc, id_vars=['Item Code', 'Description', 'UOM', 'RM Category', 'Type',
                                        'Ingredient Item', 'Description.1', 'UOM.1', 'Con_rate', 'FactCode',
                                        'Required Qty'], var_name='Month', value_name='Budgeted_Cost')

    #KFC unpivot
    up_df_kfc = pd.melt(df_kfc, id_vars=['Item Code', 'Description', 'UOM', 'RM Category', 'Type',
                                        'Ingredient Item', 'Description.1', 'UOM.1', 'Con_rate', 'FactCode',
                                        'Required Qty'], var_name='Month', value_name='Budgeted_Cost')

    # GFC Unpivot
    up_df_gfc = pd.melt(df_gfc, id_vars=['Item Code', 'Description', 'UOM', 'RM Category', 'Type',
                                        'Ingredient Item', 'Description.1', 'UOM.1', 'Con_rate', 'FactCode',
                                        'Required Qty'], var_name='Month', value_name='Budgeted_Cost')

    print('Data Unpivoted')

    #### Convert all the unpivot tables' months into month numbers
    #### NFC Data
    # Convert the date into datetime
    up_df_nfc['Month1'] = pd.to_datetime(up_df_nfc['Month'])
    # Extract the month number
    up_df_nfc['MthNum'] = up_df_nfc['Month1'].dt.month

    #### KFC Data
    # Convert the date into datetime
    up_df_kfc['Month1'] = pd.to_datetime(up_df_kfc['Month'])
    # Extract the month number
    up_df_kfc['MthNum'] = up_df_kfc['Month1'].dt.month

    #### GFC Data
    # Convert the date into datetime
    up_df_gfc['Month1'] = pd.to_datetime(up_df_gfc['Month'])
    # Extract the month number
    up_df_gfc['MthNum'] = up_df_gfc['Month1'].dt.month

    #####Join 3tables into one and then calculate the Budgeted cost per unit of required Qty
    ####### Formula to be used
    ## Budgeted cost per unit of required Qty (USD) = Budgeted cost* Con_rate/Required Qty

    #Step-1: Join the 3 tables into one
    bud_df = pd.concat([up_df_nfc, up_df_kfc, up_df_gfc], ignore_index=True)

    #Step-2: Derive Budgeted cost per unit of required Qty (USD)
    # bud_df['Bud_Cost_USD'] = (bud_df['Budgeted_Cost']*bud_df['Con_rate'])/bud_df['Required Qty']
    bud_df['Bud_Cost_USD'] = None
    print('Derive Budgeted cost per unit of required Qty (USD) - Done')

    bud_df_final =  bud_df[['FactCode','MthNum','RM Category', 'Type',
       'Ingredient Item', 'Description.1', 'UOM.1','Bud_Cost_USD']].drop_duplicates()
    
     # Create a pivot table with 'Category' as rows and 'Value' as values, and aggregate using sum
    bud_df_final1 = pd.pivot_table(bud_df_final, index=['FactCode','MthNum', 'Type',
       'Ingredient Item', 'Description.1', 'UOM.1'], values='Bud_Cost_USD', aggfunc='mean')
    ######### Extra line for 2023 data 
    bud_df_final1['Bud_Cost_USD'] = None
    ####Return the output into an excel
    bud_df_final1.head()
    bud_df_final1.to_csv('RM Budgeted cost.csv')
    print ('RM Budgeted cost conversion is Successful!!!')

except Exception as e:

       print ("RM Budgeted cost processing failed due to the following error: ", e)

Do you want to use default values? The following Conversion rates will be applied 'KFC': 0.306001, 'NFC': 3.75, 'GFC': 3.6725 (Y/N):  y


Currently used Exchange rates are : {'KFC': 3.26796317659, 'NFC': 0.26666666666, 'GFC': 0.2722940776}
Exchnage rates added
Data Unpivoted
Derive Budgeted cost per unit of required Qty (USD) - Done
RM Budgeted cost conversion is Successful!!!


In [6]:
# bud_df_final1

In [7]:
################################ Item Cost Data Transformation ##########################

try: 
    ### Convert into Pivot and add conversion rates
    # NFC pivot
    up_df_nfc = pd.pivot_table(df_nfc_i, index=['ORGANIZATION_CODE', 'PERIOD_CODE', 'ITEM_CODE', 'DESCRIPTION'], values='CMPNT_COST', aggfunc='sum')
    #KFC pivot
    up_df_kfc = pd.pivot_table(df_kfc_i, index=['ORGANIZATION_CODE', 'PERIOD_CODE', 'ITEM_CODE', 'DESCRIPTION'], values='CMPNT_COST', aggfunc='sum')
    #GFC pivot
    up_df_gfc = pd.pivot_table(df_gfc_i, index=['ORGANIZATION_CODE', 'PERIOD_CODE', 'ITEM_CODE', 'DESCRIPTION'], values='CMPNT_COST', aggfunc='sum')

    fin_df_nfc = up_df_nfc.reset_index()
    fin_df_kfc = up_df_kfc.reset_index()
    fin_df_gfc = up_df_gfc.reset_index()

    ### Take exchange rates from the input data

    conversion_rates = get_conversion_rates()
    print('Currently used Exchange rates are :', conversion_rates)

    fin_df_nfc.insert(fin_df_nfc.columns.get_loc('ORGANIZATION_CODE'), 'Con_rate', conversion_rates['NFC'])
    fin_df_kfc.insert(fin_df_kfc.columns.get_loc('ORGANIZATION_CODE'), 'Con_rate', conversion_rates['KFC'])
    fin_df_gfc.insert(fin_df_gfc.columns.get_loc('ORGANIZATION_CODE'), 'Con_rate', conversion_rates['GFC'])

    print ('Exchage rates applied')

    #### Join the 3 tables into one and then convert them into USD using 
    cons_df = pd.concat([fin_df_nfc, fin_df_kfc, fin_df_gfc], ignore_index=True)
    # cons_df['Consumption cost ($USD)'] = cons_df['CMPNT_COST']*cons_df['Con_rate']
    cons_df['Consumption cost ($USD)']= None
    cons_df.head()

    ### Export as a new output table 
    ### Export the Item cost data 
    cons_df.to_csv('Item_cost_Processed.csv',index=False)
    print('Item cost processing successful !!!')

except Exception as e:

       print ("Item cost processing failed due to the following error: ", e)


Do you want to use default values? The following Conversion rates will be applied 'KFC': 0.306001, 'NFC': 3.75, 'GFC': 3.6725 (Y/N):  y


Currently used Exchange rates are : {'KFC': 3.26796317659, 'NFC': 0.26666666666, 'GFC': 0.2722940776}
Exchage rates applied
Item cost processing successful !!!


In [8]:
# df_frct.columns

In [9]:
##################### RM Forecasted Price Processing ##################
try: 

    df_frct.columns = df_frct.columns.astype(str)

    # Adjust value_vars list to only include month columns
    # value_vars_adjusted = [
    #     'Jan-24', '2024-02-01 00:00:00', 'Mar-24', 'Apr-24', 'May-24',
    #     'Jun-24', 'Jul-24', 'Aug-24', 'Sep-24', 'Oct-24', 'Nov-24', 'Dec-24'
    # ]
    value_vars_adjusted = [
        '2023-01-01 00:00:00', '2023-02-01 00:00:00',
       '2023-03-01 00:00:00', '2023-04-01 00:00:00', '2023-05-01 00:00:00',
       '2023-06-01 00:00:00', '2023-07-01 00:00:00', '2023-08-01 00:00:00',
       '2023-09-01 00:00:00', '2023-10-01 00:00:00', '2023-11-01 00:00:00',
       '2023-12-01 00:00:00'
    ]

    # Melt the DataFrame using the adjusted value_vars list
    df2=pd.melt(df_frct, 
                        id_vars=['Status', 'ID Code', 'Code', 'Description', '2024 Volume',
                                'Entity', 'Subcategory', 'Category', 'Responsible', 
                                'Management', 'Business Category', 'Reporting Currency', 
                                'UOM-BUD', 'UOM-act', 'CORRECTION'],
                        value_vars=value_vars_adjusted,
                        var_name='Month', 
                        value_name='Value')
    
    # df3=df2.groupby(['Code','Entity','Reporting Currency','UOM-BUD','Month'])['Value'].sum().reset_index()
    # df3=df2.groupby(['Code','Entity','UOM-BUD','Month'])['Value'].sum().reset_index()
    df3=df2.groupby(['Code','Entity','Month'])['Value'].sum().reset_index()
    df3['Value'] = pd.to_numeric(df3['Value'], errors='coerce')

    # Add new column 'Value-TON revised'
    # df3['Value-TON revised'] = df3.apply(lambda row: row['Value'] / 1000 if row['UOM-BUD'] == 'TON' else row['Value'], axis=1)
    df3['Value-TON revised'] = df3['Value']
    # date_format_dict = {
    # '2024-02-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    # 'Apr-24': '%b-%y',
    # 'Aug-24': '%b-%y',
    # 'Dec-24': '%b-%y',
    # 'Jan-24': '%b-%y',
    # 'Jul-24': '%b-%y',
    # 'Jun-24': '%b-%y',
    # 'Mar-24': '%b-%y',
    # 'May-24': '%b-%y',
    # 'Nov-24': '%b-%y',
    # 'Oct-24': '%b-%y',
    # 'Sep-24': '%b-%y'
    # }
    
    date_format_dict = {
    '2023-01-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    '2023-02-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    '2023-03-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    '2023-04-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    '2023-05-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    '2023-06-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    '2023-07-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    '2023-08-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    '2023-09-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    '2023-10-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    '2023-11-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    '2023-12-01 00:00:00': '%Y-%m-%d %H:%M:%S',
    }


    # Apply the function to 'Month' column
    df3['Month'] = df3['Month'].apply(convert_to_datetime)

    # Compare unique values of the 'Entity' column in df_entity_mapping with df_frct
    unmapped_entities = df3[~df3['Entity'].isin(df_entity_mapping['Entity'])]['Entity'].unique()

    # Print the unmapped entities
    if len(unmapped_entities) > 0:
        print("These entities will be ignored:")
        print ("To map more entities please update Entity Mapping Master")
        for entity in unmapped_entities:
            print("-", entity)
            
    else:
        print("All entities are mapped in the Master Data")
    
    df_frct_final= pd.merge(df3, df_entity_mapping, on='Entity', how='inner')
    df_frct_final['Month1']= pd.to_datetime(df_frct_final['Month']).dt.month
    df_frct_final['Month1']= df_frct_final['Month1'].astype('int64')
    df_frct_final.head()

    print('Month Number conversion done')    

    df_frct_final.to_excel('forecast_cost.xlsx',index=False)
    print('Forecasted cost processing successful !!!')

except Exception as e:

       print ("Forecasted cost processing failed due to the following error: ", e)

All entities are mapped in the Master Data
Month Number conversion done
Forecasted cost processing successful !!!


#### RM Price Mapping 

Raw Material Price Mapping comprises the following stages




1. Extract FG SKU + Factory combinations to associate with the Bill of Materials (m M1.
2. Generate filtered BOM-1 and BOM-2 based on BOM availabor M4.
3. Compile a list of unique Factory-Raw Material (RM) combinations referenced in the filtered BOM.
4. Employ the list to link 12-month costs for each RM (Raw Material) from the supply base data.
5. Calculate the average cost for each RM at the facto which can be taken as an input to the Top 80% BOM calculationt for M4.

In [10]:
#### Template Files ####
#### Not Subject to change ######
####The following tables will be used as templates to check against the new input data provided and some one-time tables are also listed here #####

Capacity_template = 'Capacity_DD_Filtered.xlsx'
BOM1_template = 'BOM1.xlsx'
BOM2_template = 'BOM2.xlsx'


In [11]:
#################### Key commodity List  (Master List) - One Time Data ###################################################
# Define the data for the key commodities list- Logic confirmed from business
data = {
    'RM_Code': [1001101002, 1001101007, 1001101108, 1001102618],
    'RM_Desc': ['BRAZILIAN BEEF FOREQUARTER', 'BRAZILIAN BEEF TRIMMING', 'CHICKEN BREAST', 'KUDU CHICKEN BREAST (1*15KG)']
}

# Create a pandas DataFrame
key_commodities = pd.DataFrame(data)
# key_commodities

In [12]:
#### One time Factory Mapping (Factory Mapping Master for BOM data) #####

# def assign_factcode(row):
#     if row['COMP_ORG'] in ['NF1', 'NF2']:
#         return 'NFC'
#     elif row['COMP_ORG'] == 'GFM':
#         return 'GFC'
#     elif row['COMP_ORG'] == 'KPF':
#         return 'KFC'
#     else:
#         return None  # or whatever default value you want to assign for other cases

In [13]:
''' List of Functions Required '''
### Define Required Functions for this data process ####

#### Take input for the tables required for this data module ####
def get_table_paths():

    """
    No of Input files: 3
    If the Input file is not specified takes templates as default tables.
    
    BOM1 : Flatten BOM1 data 
    BOM2: Flatten BOM2 data
    Capacity File: File with line and throughput mapping at FG SKU level

    """  
    # Default file paths
    Capacity_template = 'Capacity_DD_Filtered.xlsx'
    BOM1_template = 'BOM1.xlsx'
    BOM2_template = 'BOM2.xlsx'
    
    # Ask if user wants to use default file paths
    use_defaults = input("Do you want to use default file paths? (Y/N): ").upper()
    
    if use_defaults == 'Y':
        
        
        capacity_file_path = Capacity_template
        BOM1_file_path = BOM1_template
        BOM2_file_path = BOM2_template
        
    else:
        
        # Prompt user to input custom file paths
        BOM1_file_path = input("Enter the path for BOM1 file : ")
        BOM2_file_path = input("Enter the path for BOM2 file : ")
        capacity_file_path = input("Enter the path for Capacity file: ")
        
           
    return BOM1_file_path,BOM2_file_path, capacity_file_path


##### Checks whether a list of columns to be verified exists within a given list of columns in a particular data frame - will be called inside 
##### 'process_dataframe' function

def column_check(to_check, columns):
    """
    Description:
    This function checks whether a list of columns to be verified exists within a given list of columns. 
    It returns a list of columns that are not present in the given list.

    Parameters:
    - to_check (list): A list of columns to be verified.
    - columns (list): A list of columns to check against.

    Returns:
    - not_present (list): A list containing columns from 'to_check' that are not present in 'columns'.
    
    Example:
    to_check = ['column1', 'column2', 'column3']
    columns = ['column1', 'column3', 'column4']
    missing_columns = column_check(to_check, columns)
    print(missing_columns)  # Output: ['column2']
    """
    not_present = []

    for i in to_check:
        if i not in columns:
            not_present.append(i)
    return not_present


##### Create a dictionary with predefined data types based on a DataFrame.- will be called inside 'check_data_types' functions

def create_predefined_types(dataframe):
    """
    Create a dictionary with predefined data types based on a DataFrame.
    
    Parameters:
    - dataframe (DataFrame): The DataFrame to create predefined types from.
    
    Returns:
    - predefined_types (dict): A dictionary where keys are column names and values are inferred data types.
    """
    predefined_types = {}
    
    for column in dataframe.columns:
        column_type = str(dataframe[column].dtype)
        predefined_types[column] = column_type
    
    return predefined_types


######### Check data types of imported data against predefined types.
def check_data_types(predefined_types, imported_data):
    """
    Check data types of imported data against predefined types.
    
    Parameters:
    - predefined_types (dict): A dictionary where keys are column names and values are predefined data types.
    - imported_data (DataFrame): The imported data to check.
    
    Returns:
    - mismatches (dict): A dictionary containing columns with data type mismatches and their corresponding types.
    """
    mismatches = {}
    
    for column, expected_type in predefined_types.items():
        imported_type = imported_data[column].dtype
        if imported_type != expected_type:
            mismatches[column] = {'Imported Type': imported_type, 'Expected Type': expected_type}

    if mismatches =={}:        
        print ('Data Type is matching')
    else :
        print ('Data Type is not matching for the following cases:')
    
    return mismatches

###### This function processes a DataFrame to check if specified columns are present. 
def process_dataframe(column_data, to_check):
    """
    Description:
    This function processes a DataFrame to check if specified columns are present. 
    It prints a message indicating whether all columns are present or lists the columns that are missing.

    Parameters:
    - column_data (DataFrame): The DataFrame to be processed.
    - to_check (list): A list of columns to check for presence in the DataFrame.

    Returns:
    None

    Example:
    column_data = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
    to_check = ['A', 'B', 'C', 'D']
    process_dataframe(column_data, to_check)
    # Output: ['D'] is not present in dataframe  DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
    """
    columns = column_data.columns
    not_present = column_check(to_check, columns)

    if len(not_present) == 0:
        print("All columns are in the dataframe")
    else:
        print(not_present, "is not present in dataframe", column_data)

In [14]:
# Load your Template Data to define column names and data types

Cap_temp = pd.read_excel(Capacity_template)
BOM1_temp = pd.read_excel(BOM1_template)
BOM2_temp = pd.read_excel(BOM2_template)

# Create predefined types of dictionary
predefined_types_cap = create_predefined_types(Cap_temp)
predefined_types_BOM1 = create_predefined_types(BOM1_temp)
predefined_types_BOM2 = create_predefined_types(BOM2_temp)


In [15]:
##### Read the Input Files ####
BOM1_file,BOM2_file, capacity_file = get_table_paths()
print("Table Paths: ", BOM1_file,BOM2_file, capacity_file)


Do you want to use default file paths? (Y/N):  y


Table Paths:  BOM1.xlsx BOM2.xlsx Capacity_DD_Filtered.xlsx


In [16]:
######################  Execute the Data Processing #########################

'''
Step - 1 : BOM Filter - Map BOM Data against Capacity-DD Mapping file and then filter for available cases, 
for the rest put retain flag as 0 

Step - 2 : Tag BOM Version to the data and combine the filtered BOM into one Data table

'''

try:
    
    #### Read the Tables 
    BOM1_file = pd.read_excel(BOM1_file)
    BOM2_file = pd.read_excel(BOM2_file)
    Capacity_file = pd.read_excel(capacity_file)

    print("Data Reading .... Done")
    
    ### Check data types and templates ####
    ### If all of these passes, then let it go to the next step####

    ## Check BOM1 data template
    process_dataframe(BOM1_file, BOM1_temp.columns)
    check_data_types(predefined_types_BOM1,BOM1_file)

    ## Check BOM2 data template
    process_dataframe(BOM2_file, BOM2_temp.columns)
    check_data_types(predefined_types_BOM2,BOM2_file)


    ## Check capacity data template
    process_dataframe(Capacity_file, Cap_temp.columns)
    check_data_types(predefined_types_cap,Capacity_file)

    print(" Data Template checking ......Done")

    #Take Capacity Base for only those where retain_flag =1
    BOM_base = Capacity_file[Capacity_file['retain_flag']==1]

    #### We need only FG+Factory mapping to pull BOM against it ####

    # Select only 'FG Code' and 'Factory' columns
    BOM_base1 = BOM_base[['FG Code', 'Factory']]

    # Drop duplicate rows based on 'FG Code' and 'Factory' columns
    BOM_base1 = BOM_base1.drop_duplicates()

    ### Add Factory Mapping to Flatten BOM data

    # BOM1_file['FactCode'] = BOM1_file.apply(assign_factcode, axis=1)
    # BOM2_file['FactCode'] = BOM2_file.apply(assign_factcode, axis=1)

    # Check BOM-1 availability based on DD-Capacity mapping 
    Filter_BOM1 = pd.merge(BOM_base1 ,BOM1_file,left_on=['Factory','FG Code'], 
                                    right_on=['FactCode','ASSEMB_ITEM_CODE'], 
                                    how='left')

    Filter_BOM1['retain_flag_BOM'] = 1  # Default value is 1
    Filter_BOM1['blank_flag_BOM'] = 0  # Default value is 0
    # Filter_BOM1.loc[(Filter_BOM1['COMP_QTY'].isnull()) | (Filter_BOM1['COMP_QTY'] == 0), 'retain_flag_BOM'] = 0
    # Updating retain flag for only blank cases and including zeros.
    Filter_BOM1.loc[(Filter_BOM1['COMP_QTY'].isnull()), 'retain_flag_BOM'] = 0
    Filter_BOM1.loc[(Filter_BOM1['COMP_QTY'] == 0), 'blank_flag_BOM'] = 1

    BOM1 = Filter_BOM1[['Factory','FG Code', 'ASSEMB_ITEM_DESC',
       'ASSEMB_UOM', 'COMP_ITEM_CODE','COMP_DESC','COMP_UOM','COMP_QTY','Packing flag','retain_flag_BOM']]
    
    if BOM1[BOM1['retain_flag_BOM'] == 1 ].empty:
        
        print ('No common records found in Demand-Capability Mapping and BOM1, please check the source data')
    else:
        print ('BOM1 filter is processed Successfully!!')

    # Check BOM-2 availability based on DD-Capacity mapping 

    # Check BOM2 availability, taken only the common cases here 

    Filter_BOM2 = pd.merge(BOM_base1,BOM2_file,left_on=['Factory','FG Code'], 
                                    right_on=['FactCode','ASSEMB_ITEM_CODE'], 
                                    how='inner')

    Filter_BOM2['retain_flag_BOM'] = 1  # Default value is 1
    Filter_BOM2['blank_flag_BOM'] = 0  # Default value is 0
    # Filter_BOM2.loc[(Filter_BOM2['COMP_QTY'].isnull()) | (Filter_BOM2['COMP_QTY'] == 0), 'retain_flag_BOM'] = 0
     # Updating retain flag for only blank cases and including zeros.
    Filter_BOM2.loc[(Filter_BOM2['COMP_QTY'].isnull()), 'retain_flag_BOM'] = 0
    Filter_BOM2.loc[(Filter_BOM1['COMP_QTY'] == 0), 'blank_flag_BOM'] = 1

    BOM2 = Filter_BOM2[['Factory','FG Code', 'ASSEMB_ITEM_DESC',
       'ASSEMB_UOM', 'COMP_ITEM_CODE','COMP_DESC','COMP_UOM','COMP_QTY','Packing flag','retain_flag_BOM']]
    
    if BOM2[BOM2['retain_flag_BOM'] == 1 ].empty:
        
        print ('No common records found in Demand-Capability Mapping and BOM2, please check the source data')
    else:
        print ('BOM2 filter is processed Successfully!!')

    ### Combine the BOM into one dataset
    BOM1['BOM_version'] = "BOM-1"
    BOM2['BOM_version'] = "BOM-2"

    ###BOM Filtered
    BOM_filtered = pd.concat([BOM1,BOM2], ignore_index=True)
    if len(BOM_filtered)>0 :
        # Check for duplicate rows based on specified columns
        duplicate_rows = BOM_filtered.duplicated(subset=['BOM_version', 'Factory', 'FG Code', 'COMP_ITEM_CODE'], keep='first')
        
        # Filter out duplicate rows, keeping only the first observation
        unique_BOM_filtered = BOM_filtered[~duplicate_rows]
        
        # Display the resulting DataFrame
        unique_BOM_filtered.to_csv('BOM Filtered.csv',index=False),
        print ('BOM Data filtered Successfully!! The data is created as- BOM Filtered.csv ')

except Exception as e:

       print ("Data Processing stopped for DD-Capacity and BOM Mapping due to the following error: ", e)


Data Reading .... Done
All columns are in the dataframe
Data Type is matching
All columns are in the dataframe
Data Type is matching
All columns are in the dataframe
Data Type is matching
 Data Template checking ......Done
BOM1 filter is processed Successfully!!
BOM2 filter is processed Successfully!!
BOM Data filtered Successfully!! The data is created as- BOM Filtered.csv 


In [17]:
cons_df.columns

Index(['Con_rate', 'ORGANIZATION_CODE', 'PERIOD_CODE', 'ITEM_CODE',
       'DESCRIPTION', 'CMPNT_COST', 'Consumption cost ($USD)'],
      dtype='object')

In [18]:
# ######################  Execute the Data Processing #########################
# '''
# Step - 3: Derive Only Retained BOM Data
# Step - 4: Prepare a List of RMs that are being used in the retained BOM data
# Step - 5: Map Key Item's RM Cost, and other RM Costs
# Step - 6: QC For RM costs

# '''

# try:
#     ############################## Prep base RM List ##################################################

#     BOM_retained = pd.concat([BOM1[BOM1['retain_flag_BOM']==1],BOM2[BOM2['retain_flag_BOM']==1]], ignore_index=True)
#     # Selecting only the required columns
#     RM_Used = BOM_retained[['Factory', 'COMP_ITEM_CODE']]
#     # Dropping duplicates
#     RM_Used = RM_Used.drop_duplicates()
#     if len(RM_Used)>0:
#           print ('RM List prepared Successfully!!')
#     else:
#           print ('RM list is not prepared properly. Please check the input data')
    
#     RM_list = RM_Used
#     RM_list['temp'] = 1

#     ########### List of months -- Since we want to map the RM cost data for all the months ###########
#     data = {'MthNum': range(1, 13)}  # Generates numbers from 1 to 12
#     MthNum = pd.DataFrame(data)
#     MthNum['temp']=1

#     # Perform a full outer join on the 'temp' column
#     RM_Base = pd.merge(RM_list, MthNum, on='temp', how='outer').drop(columns=['temp'])
#     RM_Base.head()

#     #Check for counts
#     #### Need to add error handling properly#####################
   

#     if len(RM_Base) != len(RM_list)*len(MthNum):
#         print("Count mismatch error at Facotry-RM-Month level: Stopping the process...")
        
#     else:            
#         print ('Factory-RM-Month Base list .....Done')

#     '''

#     RM-PM Cost Logic to be used :

#     1. For key commodities-
#         a) for first 3 months, the consumption price will be applied
#         b) for rest of the 10 months, the forcasted price will be applied 
#     2. For other RMs for all the months forcasted price to be taken
#     3. For all the cases, where ever both the costs are missing we have to use budgeted cost

#     '''
#     ###################### Cost Mapping For Key Commodities #####################################

#     ## Filter Data based on Key commodities

#     # List of entities from key_commodities DataFrame
#     key_entities = key_commodities['RM_Code'].tolist()

#     # Filtering data based on the key commodities
#     filter_key = RM_Base[RM_Base['COMP_ITEM_CODE'].isin(key_entities)]
#     filter_key.rename(columns={'Factory': 'FACTCode'}, inplace=True)
#     print (" The following RMs has been considered as Key Commodity, to add more please update the Key Commodity Master list", filter_key['COMP_ITEM_CODE'].unique())

#     ##### Define Forecast cost data
#     frcst_cost = df_frct_final.copy()

#     ##### Define Item Cost data
#     item_cost = cons_df.copy()

#     ### Define  Budgeted cost
#     bud_cost = bud_df_final1.copy()
     
    
#     # Merge with item_cost DataFrame for the first 3 months

#     # merged_first_3_months = pd.merge(filter_key[filter_key['MthNum'].isin([1,2,3])], item_cost, 
#     #                                 left_on=['FACTCode', 'COMP_ITEM_CODE'], 
#     #                                 right_on=['ORGANIZATION_CODE', 'ITEM_CODE'], 
#     #                                 how='left')

#     # print('Consumption Cost is mapped')
#     ############ Map Forecasted cost for key commodities as Consumption cost is missing

#     merged_first_3_months = pd.merge(filter_key[filter_key['MthNum'].isin([1,2,3])], frcst_cost, 
#                                     left_on=['FACTCode', 'COMP_ITEM_CODE', 'MthNum'], 
#                                     right_on=['Factory code', 'Code', 'Month1'], 
#                                     how='left')

#     print('Consumption Cost is mapped')
    

#     # Merge with frcst_cost DataFrame for the remaining months
#     merged_remaining_months = pd.merge(filter_key[~filter_key['MthNum'].isin([1,2,3])], frcst_cost, 
#                                     left_on=['FACTCode', 'COMP_ITEM_CODE', 'MthNum'], 
#                                     right_on=['Factory code', 'Code', 'Month1'], 
#                                     how='left')
    
#     if len(merged_remaining_months[merged_remaining_months['Value-TON revised']>0])>0: 
#          print ('Forecasted cost mapped for key commodities')
#     else:
#          print('Forecasted cost is not available for Key commodities')

#     # Key_three_months= merged_first_3_months[['FACTCode', 'COMP_ITEM_CODE', 'MthNum','Consumption cost ($USD)']]
#     # # Renaming columns
#     # Key_three_months = Key_three_months.rename(columns={
#     #     'Consumption cost ($USD)': 'RM Cost'
#     # })
#     Key_three_months = merged_first_3_months[['FACTCode', 'COMP_ITEM_CODE', 'MthNum','Value-TON revised']]
#     Key_remaining_months = merged_remaining_months[['FACTCode', 'COMP_ITEM_CODE', 'MthNum','Value-TON revised']]
#     # Renaming columns
#     Key_remaining_months = Key_remaining_months.rename(columns={
#         'Value-TON revised': 'RM Cost'
#     })
#     # Renaming columns
#     Key_three_months = Key_remaining_months.rename(columns={
#         'Value-TON revised': 'RM Cost'
#     })
#     key_cost = pd.concat([Key_three_months, Key_remaining_months], ignore_index=True)
#     # key_cost  
#     # Set 'retain_flag' column to 1 where 'RM Cost' is greater than 0
#     key_cost.loc[key_cost['RM Cost'] > 0, 'retain_flag'] = 1

#     ###################### Cost Mapping For Other RM and PMs #####################################

#     # Filtering data based on Non-Key Commodities
#     filter_rest = RM_Base[~RM_Base['COMP_ITEM_CODE'].isin(key_entities)]
#     filter_rest['COMP_ITEM_CODE'].unique() 
#     filter_rest.rename(columns={'Factory': 'FACTCode'}, inplace=True)

#     # Merge with frcst_cost DataFrame for the remaining months
#     filter_rest_cost = pd.merge(filter_rest, frcst_cost, 
#                                     left_on=['FACTCode', 'COMP_ITEM_CODE', 'MthNum'], 
#                                     right_on=['Factory code', 'Code', 'Month1'], 
#                                     how='left')
#     if len(filter_rest_cost[filter_rest_cost['Value-TON revised']>=0])>0:
        
#         print("Forcasted Cost is mapped for non Key commodities")

#     else:
         
#         print ("Forcasted cost unavilable please check the input data")

#     ##### Rename The columns #######
        
#     filter_rest_cost = filter_rest_cost[['FACTCode', 'COMP_ITEM_CODE', 'MthNum','Value-TON revised']]
#     # Renaming columns
#     filter_rest_cost = filter_rest_cost.rename(columns={
#         'Value-TON revised': 'RM Cost'
#     })
#     # filter_rest_cost

#     ####################  Get Unmapped parts from each dataset #############################################

#     ## If there is blank or zero after mapping with consumption cost and forcasted cost, will be taken as unmapped and will be mapped against Budgeted Numbers

#     # key_cost_unmapped = key_cost[(key_cost['RM Cost'] <= 0) | (key_cost['RM Cost'].isnull())]
#     # key_cost_mapped = key_cost[(key_cost['RM Cost']>0)]
#     # key_cost['retain flag'] = 1

#     # key_cost_unmapped.head()


#     filter_rest_cost_unmapped = filter_rest_cost[(filter_rest_cost['RM Cost'] <= 0) | (filter_rest_cost['RM Cost'].isnull())]
#     filter_rest_cost_mapped = filter_rest_cost[(filter_rest_cost['RM Cost'] > 0)]
#     filter_rest_cost_mapped['retain flag'] = 1


#     # filter_rest_cost_unmapped

#     ## Check the counts after splitting
#     ######### Need to add effor handling properly ###############
             
#     if (len(filter_rest_cost)==len(filter_rest_cost_unmapped)+len(filter_rest_cost_mapped)):
        
#         print("Unmapped cases splitting sucessful !!")
    
#     else :
            
#          print ("Count Mismatch after splitting - Error")

   

#     # ###Map the unmapped cases with the budgeted cost
#     # key_cost_unmapped_bud = pd.merge(key_cost_unmapped [['FACTCode', 'COMP_ITEM_CODE', 'MthNum']], bud_cost, 
#     #                                 left_on=['FACTCode', 'COMP_ITEM_CODE', 'MthNum'], 
#     #                                 right_on=['FactCode', 'Ingredient Item', 'MthNum'], 
#     #                                 how='left')
#     # key_cost_unmapped_bud = key_cost_unmapped_bud[['FACTCode', 'COMP_ITEM_CODE', 'MthNum','Bud_Cost_USD']]
#     # key_cost_unmapped_bud['retain flag'] = 0 # Set default value to 0
#     # key_cost_unmapped_bud.loc[(key_cost_unmapped_bud['Bud_Cost_USD'].notnull())& (key_cost_unmapped_bud['Bud_Cost_USD'] != 0), 'retain flag'] = 1
#     # # Renaming columns
#     # key_cost_unmapped_bud = key_cost_unmapped_bud.rename(columns={
#     #     'Bud_Cost_USD': 'RM Cost'
#     # })
#     # # key_cost_unmapped_bud       

#     ##RM Cost for rest things

#     filter_rest_cost_unmapped_bud = pd.merge(filter_rest_cost_unmapped [['FACTCode', 'COMP_ITEM_CODE', 'MthNum']], bud_cost, 
#                                     left_on=['FACTCode', 'COMP_ITEM_CODE', 'MthNum'], 
#                                     right_on=['FactCode', 'Ingredient Item', 'MthNum'], 
#                                     how='left')
#     filter_rest_cost_unmapped_bud = filter_rest_cost_unmapped_bud[['FACTCode', 'COMP_ITEM_CODE', 'MthNum','Bud_Cost_USD']]
#     filter_rest_cost_unmapped_bud['retain flag'] = 0  # Set default value to 0
#     filter_rest_cost_unmapped_bud.loc[(filter_rest_cost_unmapped_bud['Bud_Cost_USD'].notnull())&(filter_rest_cost_unmapped_bud['Bud_Cost_USD']!=0), 'retain flag'] = 1
#     # Renaming columns
#     filter_rest_cost_unmapped_bud = filter_rest_cost_unmapped_bud.rename(columns={
#         'Bud_Cost_USD': 'RM Cost'
#     })
#     # filter_rest_cost_unmapped_bud

#     # Combine all the RM cost into one table 
#     RM_Cost_mapped_old = pd.concat([key_cost,filter_rest_cost_mapped,filter_rest_cost_unmapped_bud], ignore_index=True)
    
#     RM_Cost_mapped_old.to_csv(r'RM_Cost_mapped_old.csv', index=False)      


# except Exception as e:

#        print ("RM Cost mapping process failed due to the following error: ", e)


In [19]:
######################  Execute the Data Processing #########################
'''
Step - 3: Derive Only Retained BOM Data
Step - 4: Prepare a List of RMs that are being used in the retained BOM data
Step - 5: Map Key Item's RM Cost, and other RM Costs
Step - 6: QC For RM costs

'''

try:
    ################################### RM Cost Old derivation ####################################
    BOM_retained = pd.concat([BOM1[BOM1['retain_flag_BOM']==1],BOM2[BOM2['retain_flag_BOM']==1]], ignore_index=True)
    # Selecting only the required columns
    RM_Used = BOM_retained[['Factory', 'COMP_ITEM_CODE']]
    # Dropping duplicates
    RM_Used = RM_Used.drop_duplicates()
    if len(RM_Used)>0:
        print ('RM List prepared Successfully!!')
    else:
        print ('RM list is not prepared properly. Please check the input data')

    RM_list = RM_Used
    RM_list['temp'] = 1

    ########### List of months -- Since we want to map the RM cost data for all the months ###########
    data = {'MthNum': range(1, 13)}  # Generates numbers from 1 to 12
    MthNum = pd.DataFrame(data)
    MthNum['temp']=1

    # Perform a full outer join on the 'temp' column
    RM_Base = pd.merge(RM_list, MthNum, on='temp', how='outer').drop(columns=['temp'])
    RM_Base.head()

    #Check for counts
    #### Need to add error handling properly#####################


    if len(RM_Base) != len(RM_list)*len(MthNum):
        print("Count mismatch error at Facotry-RM-Month level: Stopping the process...")
        
    else:            
        print ('Factory-RM-Month Base list .....Done')


    '''

    RM-PM Cost Logic to be used :

    For all the RMs only forcasted price will be mapped

    '''
    ##### Define Forecast cost data
    frcst_cost = df_frct_final.copy()
    mapped_cost =  pd.merge(RM_Base, frcst_cost, 
                                    left_on=['Factory', 'COMP_ITEM_CODE', 'MthNum'], 
                                    right_on=['Factory code', 'Code', 'Month1'], 
                                    how='left')
    mapped_cost['retain_flag']=0
    # Set 'retain_flag' column to 1 where 'RM Cost' is greater than 0
    mapped_cost.loc[mapped_cost['Value-TON revised'] > 0, 'retain_flag'] = 1
    # mapped_cost.columns
    mapped_cost = mapped_cost[['Factory', 'COMP_ITEM_CODE', 'MthNum','Value','retain_flag']]
    mapped_cost = mapped_cost.rename(columns={
        'Factory': 'FACTCode',
        'COMP_ITEM_CODE': 'COMP_ITEM_CODE',
        'MthNum': 'MthNum',
        'Value': 'RM Cost',
        'retain_flag': 'retain_flag'
    })
    mapped_cost
    mapped_cost.to_csv(r'RM_Cost_mapped_old.csv', index=False)  

except Exception as e:

    print ("RM Cost mapping process failed due to the following error: ", e)

RM List prepared Successfully!!
Factory-RM-Month Base list .....Done


In [20]:
####################### Adding Back missing Cost ##########################################################
try:

    ##### New RM costs are added in 'Average RM Cost New' . it contains old data  and new data from missing file
        
    RM_Cost_mapped= pd.read_csv(r'RM_Cost_mapped_old+missing.csv')
    
    
    #####################################################################
    print (" Monthly RM cost is prepared and generated !!!! QC pending .......")  

    '''
    QC Check : RM landed cost does not vary more than 5% month-on-month for any RM

    Note: The QC will only be applied for the RM-Factory Combinations that has landed cost available for all the months - IF QC is failed a report will be generated for failed cases

    '''   

    # Filter rows where retain flag is equal to 1
    filtered_RM_Cost_mapped = RM_Cost_mapped[RM_Cost_mapped['retain flag'] == 1]

    # Group by FACTCode and COMP_ITEM_CODE, and count the number of unique MthNum values
    grouped_counts = filtered_RM_Cost_mapped.groupby(['FACTCode', 'COMP_ITEM_CODE'])['MthNum'].nunique()

    # Filter groups where the count of unique MthNum values is 12 (for all 12 months)
    valid_groups = grouped_counts[grouped_counts == 12].index

    # Apply the check for RM landed cost variation for the valid groups
    violations = []

    # Define column names
    columns = ['FACTCode', 'COMP_ITEM_CODE', 'MthNum', 'RM Cost', 'retain flag', 'RM Cost Change']
    ## Define Blank Dataframe for QC failed cases 
    QC_failed = pd.DataFrame(columns=columns)

    for fact_code, comp_item_code in valid_groups:
        group_data = filtered_RM_Cost_mapped[(filtered_RM_Cost_mapped['FACTCode'] == fact_code) & (filtered_RM_Cost_mapped['COMP_ITEM_CODE'] == comp_item_code)]
        group_data['RM Cost Change'] = group_data['RM Cost'].pct_change() * 100
        # Set percentage change as zero for the first month
        group_data.loc[group_data['MthNum'] == group_data['MthNum'].min(), 'RM Cost Change'] = 0
        if group_data['RM Cost Change'].abs().max() > 5:
            violations.append((fact_code, comp_item_code))
            QC_failed = QC_failed._append(group_data,ignore_index=True)

    ### Export a report of RMs that is failing this perticular QC 

    QC_report_path = 'QC_failed.csv'
    QC_failed.to_csv(QC_report_path,index=False)

    # List down COMP_ITEM_CODE values for which the check is not applied
    not_applied = grouped_counts[grouped_counts < 12].index

    print("RMs where the check is not applied:")
    print(not_applied)

    if violations:
        print("RM landed cost varies more than 5% month-on-month for RMs added in the report at:",QC_report_path )
        
    else:
        print("RM landed cost does not vary more than 5% month-on-month for any RM.")


except Exception as e:

       print ("RM Cost mapping process failed due to the following error: ", e)

 Monthly RM cost is prepared and generated !!!! QC pending .......
RMs where the check is not applied:
MultiIndex([], names=['FACTCode', 'COMP_ITEM_CODE'])
RM landed cost varies more than 5% month-on-month for RMs added in the report at: QC_failed.csv


In [21]:
######################  Execute the Data Processing #########################
'''
Step-7: Derive Average RM cost for only those cases where Factory-RM level cost is available for all the months and return the average cost data

'''

try:
       
    #### Take sum of retain flag ###
    df2 = RM_Cost_mapped.groupby(['FACTCode', 'COMP_ITEM_CODE'])['retain flag'].sum()
    df2 = pd.DataFrame(df2)

    # Merge RM_Cost_mapped with df2 based on 'FACTCode' and 'COMP_ITEM_CODE'
    merged_df = pd.merge(RM_Cost_mapped, df2, on=['FACTCode', 'COMP_ITEM_CODE'], suffixes=('_RM_Cost_mapped', '_df2'))

    # Filter merged_df for rows where retain flag is 12 (As there are total of 12 months)
    filtered_RM_Cost_mapped = merged_df[merged_df['retain flag_df2'] == 12]
    filtered_RM_Cost_mapped
    print('filtered_RM_Cost_mapped is ready')
    
    # Group by 'FACTCode' and 'COMP_ITEM_CODE' and calculate the mean of 'RM Cost' within each group
    average_RM_cost = filtered_RM_Cost_mapped.groupby(['FACTCode', 'COMP_ITEM_CODE'])['RM Cost'].mean().reset_index()
    average_RM_cost['retain_flag'] = 12
    # average_RM_cost

    # Display the resulting DataFrame
    average_RM_cost1 = average_RM_cost.copy()
    # df.groupby(['col1', 'col2']).size().reset_index(name='counts')
    average_RM_cost1 = average_RM_cost1.groupby(['FACTCode', 'COMP_ITEM_CODE']).size().reset_index(name='counts')
    if len(average_RM_cost1[average_RM_cost1['counts']>1])== 0:
          print("No duplicate entries found for 'FACTCode' and 'COMP_ITEM_CODE' combinations.")
          average_RM_cost.to_csv('Average RM Cost.csv',index=False)
          print( "Average RM Cost derivation is Done!!! ")
    else:
        duplicate_entries = average_RM_cost1[average_RM_cost1['counts']>1]
        print("Duplicate entries found for the following 'FACTCode' and 'COMP_ITEM_CODE' combinations: (please check the input data)")
        print(duplicate_entries[['FACTCode', 'COMP_ITEM_CODE']].drop_duplicates())
        print( "Average RM Cost derivation is Stopped..... ")


except Exception as e:

       print ("RM Cost mapping process failed due to the following error: ", e)



filtered_RM_Cost_mapped is ready
No duplicate entries found for 'FACTCode' and 'COMP_ITEM_CODE' combinations.
Average RM Cost derivation is Done!!! 
